### Volume Trend Line Chart (Dashboard 1)

In [5]:
import pandas as pd
from datetime import datetime, timedelta

def generate_volume_trend(view="24h"):
    # Load CSVs
    solver_df = pd.read_csv("data/solver_competitions.csv")
    tx_df = pd.read_csv("data/transaction.csv")

    # Merge on auctionId
    merged = pd.merge(solver_df, tx_df, on="auctionId", how="inner")

    # Compute volume
    merged["volume"] = merged[["buyAmountInUSD", "sellAmountInUSD"]].min(axis=1)
    merged["timestamp"] = pd.to_datetime(merged["timestamp"])

    # Filter by time range
    now = merged["timestamp"].max()  # Use latest timestamp in data
    if view == "24h":
        cutoff = now - timedelta(hours=24)
        freq = "1h"
    elif view == "7d":
        cutoff = now - timedelta(days=7)
        freq = "4h"
    else:
        cutoff = merged["timestamp"].min()
        freq = "1H"  # default bin size

    merged = merged[merged["timestamp"] >= cutoff]

    # Bin by time
    merged["time_bin"] = merged["timestamp"].dt.floor(freq)

    # Aggregate volume
    trend = merged.groupby("time_bin")["volume"].sum().reset_index()
    trend["time_bin"] = trend["time_bin"].dt.strftime("%Y-%m-%d %H:%M")
    trend.rename(columns={"time_bin": "timestamp"}, inplace=True)

    # Save to JSON
    output_file = f"out/volume_trend_{view}.json"
    trend.to_json(output_file, orient="records")
    print(f"Wrote {output_file}")

generate_volume_trend("24h")
generate_volume_trend("7d")

Wrote out/volume_trend_24h.json
Wrote out/volume_trend_7d.json


### Order Solver Time Difference (Dashboard 1)

In [13]:
import pandas as pd
from datetime import datetime
from pathlib import Path

def generate_order_solver_time_diff(
    solver_path="data/solver_competitions.csv",
    tx_path="data/transaction.csv",
    order_path="data/order.csv",
    output_path="out/order_solver_time_diff.json"
):
    # Load data
    solver_df = pd.read_csv(solver_path)
    tx_df = pd.read_csv(tx_path)
    order_df = pd.read_csv(order_path)

    # 🛠 Fix: clean timestamps before merge
    tx_df["timestamp"] = pd.to_datetime(tx_df["timestamp"], utc=True).dt.tz_localize(None)
    order_df["createdTimestamp"] = pd.to_datetime(order_df["createdTimestamp"], utc=True).dt.tz_localize(None)

    # Merge
    merged = pd.merge(solver_df, tx_df, on="auctionId", how="inner")
    merged = pd.merge(merged, order_df, left_on="orderId", right_on="id", how="inner")

    # Compute time difference in seconds
    merged["time_diff_sec"] = (merged["timestamp"] - merged["createdTimestamp"]).dt.total_seconds()
    merged = merged.dropna(subset=["time_diff_sec"])

    # Filter between 0–180s
    merged = merged[(merged["time_diff_sec"] >= 0) & (merged["time_diff_sec"] <= 180)]

    # Bin into ranges (10s buckets)
    bins = list(range(0, 190, 10))
    labels = [f"{i}-{i+10}" for i in bins[:-1]]
    merged["bucket"] = pd.cut(merged["time_diff_sec"], bins=bins, labels=labels, right=False)

    # Count per bucket
    result = merged.groupby("bucket", observed=True).size().reset_index(name="count")
    result = result.dropna()
    result["bucket"] = result["bucket"].astype(str)

    # Save output
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    result.to_json(output_path, orient="records", indent=2)
    print(f"[✓] Wrote {len(result)} bins to {output_path}")

# Run
generate_order_solver_time_diff()


[✓] Wrote 14 bins to out/order_solver_time_diff.json


### Solver Participation % (Dashboard 1)

In [21]:
import pandas as pd
from pathlib import Path

def generate_solver_participation(
    participation_path="data/solver_competitions.csv",
    solver_labels_path="data/solver.csv",
    output_path="out/solver_participation.json"
):
    df = pd.read_csv(participation_path)
    label_df = pd.read_csv(solver_labels_path)

    # Drop missing
    df = df.dropna(subset=["solverAddress", "auctionId"])

    # Total auctions
    total_auctions = df["auctionId"].nunique()

    # Unique auctions per solver
    participation = df.groupby("solverAddress")["auctionId"].nunique().reset_index(name="auctions")
    participation["participation_pct"] = (participation["auctions"] / total_auctions * 100).round(2)

    # Merge with labelName
    participation = pd.merge(participation, label_df, left_on="solverAddress", right_on="address", how="left")
    participation["name"] = participation["labelName"].fillna(participation["solverAddress"])

    # Sort and limit
    participation = participation.sort_values(by="participation_pct", ascending=False).head(20)

    # Rename for frontend
    participation = participation[["solverAddress", "name", "participation_pct"]]
    participation = participation.rename(columns={"solverAddress": "solver"})

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    participation.to_json(output_path, orient="records", indent=2)
    print(f"[✓] Wrote {len(participation)} labeled solvers to {output_path}")

generate_solver_participation()

[✓] Wrote 20 labeled solvers to out/solver_participation.json


### Token Pair Treemap

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

def generate_token_pair_treemap_with_bins(
    comp_path="data/solver_competitions.csv",
    order_path="data/order.csv",
    token_labels_path="data/token_labels.csv",
    output_path="out/treemap_token_pair_volume.json"
):
    comp_df = pd.read_csv(comp_path)
    order_df = pd.read_csv(order_path)
    label_df = pd.read_csv(token_labels_path, on_bad_lines="skip")

    order_df = order_df.rename(columns={
        "buyAmountInUSD": "orderBuyUSD",
        "sellAmountInUSD": "orderSellUSD",
    })

    # Keep only relevant columns
    label_df = label_df[["address", "symbol"]]
    merged = pd.merge(comp_df, order_df, left_on="orderId", right_on="id", how="inner")
    merged = pd.merge(merged, label_df, left_on="sellTokenAddress", right_on="address", how="left").rename(columns={"symbol": "sellSymbol"})
    merged = pd.merge(merged, label_df, left_on="buyTokenAddress", right_on="address", how="left").rename(columns={"symbol": "buySymbol"})

    merged = merged.dropna(subset=["sellSymbol", "buySymbol", "buyAmountInUSD", "sellAmountInUSD"])
    merged["tokenPair"] = merged.apply(
        lambda row: "-".join(sorted([row["sellSymbol"], row["buySymbol"]])),
        axis=1
    )
    merged["volume"] = merged[["buyAmountInUSD", "sellAmountInUSD"]].min(axis=1)

    # Bin volumes
    bins = [0, 1000] + [1000 + i * 1000 for i in range(1, 11)] + [np.inf]
    labels = [f"{int(bins[i])}-{int(bins[i+1]) if bins[i+1] != np.inf else '10000+'}" for i in range(len(bins) - 1)]
    merged["volume_bin"] = pd.cut(merged["volume"], bins=bins, labels=labels, right=False)

    # Group by tokenPair
    grouped = merged.groupby("tokenPair")

    output = []
    for pair, group in grouped:
        bin_counts = group["volume_bin"].value_counts().reindex(labels, fill_value=0).to_dict()
        total_volume = group["volume"].sum()
        output.append({
            "tokenPair": pair,
            "value": total_volume,
            "binCounts": bin_counts
        })

    # Sort and keep top 100
    output = sorted(output, key=lambda x: x["value"], reverse=True)[:100]

    # Save
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    pd.DataFrame(output).to_json(output_path, orient="records", indent=2)
    print(f"[✓] Saved treemap data with bin counts to {output_path}")

generate_token_pair_treemap_with_bins()

[✓] Saved treemap data with bin counts to out/treemap_token_pair_volume.json


### Surplus Line (Dashboard 2)

In [13]:
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

def generate_surplus_trend(
    solver_address,
    from_token,
    to_token,
    view="7d",  # or "24h"
    competition_path="data/solver_competitions.csv",
    order_path="data/order.csv",
    token_labels_path="data/token_labels.csv",
    output_path_template="out/surplus_trend_{solver}_{fromtoken}_{totoken}_{view}.json"
):
    ...

    # Load data
    df = pd.read_csv(competition_path)
    orders = pd.read_csv(order_path)
    labels = pd.read_csv(token_labels_path, on_bad_lines="skip")[["address", "symbol"]]

    # Filter by solver
    df = df[df["solverAddress"] == solver_address]

    # Merge with order info for timestamp and token addresses
    df = pd.merge(df, orders[["id", "createdTimestamp", "sellTokenAddress", "buyTokenAddress"]],
                  left_on="orderId", right_on="id", how="left")
    df["createdTimestamp"] = pd.to_datetime(df["createdTimestamp"], errors="coerce")

    # Filter time range
    # now = pd.Timestamp.utcnow()
    now = df["createdTimestamp"].max()
    if view == "24h":
        df = df[df["createdTimestamp"] >= now - timedelta(days=1)]
    elif view == "7d":
        df = df[df["createdTimestamp"] >= now - timedelta(days=7)]
    else:
        raise ValueError("view must be '24h' or '7d'")



    # Map token addresses to symbols
    df = pd.merge(df, labels, left_on="sellTokenAddress", right_on="address", how="left").rename(columns={"symbol": "sellSymbol"})
    df = pd.merge(df, labels, left_on="buyTokenAddress", right_on="address", how="left").rename(columns={"symbol": "buySymbol"})

    # Clean and filter
    df = df.dropna(subset=["createdTimestamp", "surplus", "sellSymbol", "buySymbol"])
    df = df[(df["sellSymbol"] == from_token) & (df["buySymbol"] == to_token)]

    # After final filtering
    df = df[(df["sellSymbol"] == from_token) & (df["buySymbol"] == to_token)]

    # Format output
    df = df.sort_values("createdTimestamp")
    df["createdTimestamp"] = df["createdTimestamp"].dt.strftime("%Y-%m-%d %H:%M")
    df["tokenPair"] = df.apply(lambda row: f"{row['sellSymbol']} → {row['buySymbol']}", axis=1)

    result = df[["createdTimestamp", "sellSymbol", "buySymbol", "tokenPair", "surplus"]]

    # Write file
    Path("out").mkdir(parents=True, exist_ok=True)
    from_token_safe = from_token.replace("/", "-")
    to_token_safe = to_token.replace("/", "-")
    out_path = output_path_template.format(
        solver=solver_address[-6:], fromtoken=from_token.replace("/", "-"), totoken=to_token.replace("/", "-"), view=view
    )

    result.to_json(out_path, orient="records", indent=2)
    print(f"[✓] Surplus trend written to {out_path} with {len(result)} entries.")


generate_surplus_trend(
    solver_address="0x00806daa2cfe49715ea05243ff259deb195760fc",
    from_token="Wrapped Ether",
    to_token="USD Coin"
)




[✓] Surplus trend written to out/surplus_trend_5760fc_Wrapped Ether_USD Coin_7d.json with 489 entries.


### Bubble Chart (Dashboard 2)

In [15]:
import pandas as pd
import numpy as np
from pathlib import Path

def generate_solver_bubble_data(
    solver_address,
    token_labels_path="data/token_labels.csv",
    order_path="data/order.csv",
    competition_path="data/solver_competitions.csv",
    output_path="out/bubble_solver_volume_{solver}.json"
):
    # Load input files
    comp_df = pd.read_csv(competition_path)
    order_df = pd.read_csv(order_path)
    labels = pd.read_csv(token_labels_path, on_bad_lines="skip")[["address", "symbol"]]

    order_df = order_df.rename(columns={
        "buyAmountInUSD": "orderBuyUSD",
        "sellAmountInUSD": "orderSellUSD",
    })

    # Filter by solver
    comp_df = comp_df[comp_df["solverAddress"] == solver_address]

    # Merge orders with competition data
    merged = pd.merge(
        comp_df,
        order_df,
        left_on="orderId",
        right_on="id",
        how="inner"
    )

    # Add token labels
    merged = pd.merge(
        merged, labels, left_on="sellTokenAddress", right_on="address", how="left"
    ).rename(columns={"symbol": "sellSymbol"})
    merged = pd.merge(
        merged, labels, left_on="buyTokenAddress", right_on="address", how="left"
    ).rename(columns={"symbol": "buySymbol"})

    # Drop missing values
    merged = merged.dropna(subset=["sellSymbol", "buySymbol", "buyAmountInUSD", "sellAmountInUSD"])

    # Create normalized token pair name
    merged["tokenPair"] = merged.apply(
        lambda row: "-".join(sorted([row["sellSymbol"], row["buySymbol"]])),
        axis=1
    )

    # Compute per-row volume
    merged["volume"] = merged[["buyAmountInUSD", "sellAmountInUSD"]].min(axis=1)

    # Bin definitions (same as treemap)
    bins = [0, 1000] + [1000 + i * 1000 for i in range(1, 11)] + [np.inf]
    labels_bins = [
        f"{int(bins[i])}-{int(bins[i + 1]) if bins[i + 1] != np.inf else '10000+'}"
        for i in range(len(bins) - 1)
    ]
    merged["volume_bin"] = pd.cut(merged["volume"], bins=bins, labels=labels_bins, right=False)

    # Group by token pair
    bubble_data = []
    for token_pair, group in merged.groupby("tokenPair"):
        bin_counts = group["volume_bin"].value_counts().reindex(labels_bins, fill_value=0).to_dict()
        total_volume = group["volume"].sum()

        bubble_data.append({
            "tokenPair": token_pair,
            "value": total_volume,
            "binCounts": bin_counts
        })

    # Keep top 50 by volume
    bubble_data = sorted(bubble_data, key=lambda x: x["value"], reverse=True)[:50]

    # Save output
    suffix = solver_address[-6:]
    output_file = output_path.format(solver=suffix)
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    pd.DataFrame(bubble_data).to_json(output_file, orient="records", indent=2)

    print(f"[✓] Saved {len(bubble_data)} token pairs with volume + binCounts to {output_file}")

generate_solver_bubble_data("0x00806daa2cfe49715ea05243ff259deb195760fc")

[✓] Saved 50 token pairs with volume + binCounts to out/bubble_solver_volume_5760fc.json


### Solver Filled Donut Chart (Dashboard 2)

In [17]:
import pandas as pd
from pathlib import Path
import json

def generate_partially_fillable_donut_data(
    solver_address,
    order_path="data/order.csv",
    competition_path="data/solver_competitions.csv",
    output_path="out/partially_fillable_{solver}.json"
):
    # Load input files
    comp_df = pd.read_csv(competition_path)
    order_df = pd.read_csv(order_path)

    # Filter by solver
    comp_df = comp_df[comp_df["solverAddress"] == solver_address]

    # Merge with order data to get partiallyFillable field
    merged = pd.merge(
        comp_df,
        order_df[["id", "partiallyFillable"]],
        left_on="orderId",
        right_on="id",
        how="inner"
    )

    # Convert to bool and count
    merged["partiallyFillable"] = merged["partiallyFillable"].astype(bool)
    counts = merged["partiallyFillable"].value_counts().to_dict()

    # Match format: key = label, value = count
    binCounts = {
        "True": counts.get(True, 0),
        "False": counts.get(False, 0)
    }

    # Save output
    suffix = solver_address[-6:]
    output_file = output_path.format(solver=suffix)
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)

    with open(output_file, "w") as f:
        json.dump({"binCounts": binCounts}, f, indent=2)

    print(f"[✓] Saved partiallyFillable donut data for solver {solver_address} to {output_file}")

# Example usage
generate_partially_fillable_donut_data("0x00806daa2cfe49715ea05243ff259deb195760fc")


[✓] Saved partiallyFillable donut data for solver 0x00806daa2cfe49715ea05243ff259deb195760fc to out/partially_fillable_5760fc.json
